In [1]:
%run ./library/normal_distributions_and_likelihoods.py
%run ../utils/helper.py

#### Calculate Likelihoods for Distributions and Points

In [2]:
html(calc_likelihoods_for_distributions_and_points)

In [3]:
points = np.array([[ 2.67536762,  1.61771433],
        [-1.51995242,  8.5575738 ],
        [ 5.9708441 ,  2.88736993],
        [-0.26015255, -1.21744485],
        [-4.24165224,  3.16340135],
        [10.58514974, -1.02516812],
        [11.83292385, -1.69103837],
        [ 2.55579144,  3.80067187],
        [ 1.15372272, -2.601238  ],
        [ 6.47700649,  8.65276974]])

distributions = np.array([
    [[3, 4], [1, 0], [0, 1]],
    [[2, 0], [1, 0], [0, 1]]
])

calc_likelihoods_for_distributions_and_points(distributions, points)

array([[8.84226101e-03, 3.42373678e-02],
       [1.79903328e-10, 4.06639803e-20],
       [1.03867726e-03, 9.28095634e-07],
       [9.61006664e-10, 5.89820140e-03],
       [4.59522127e-13, 3.70794812e-12],
       [1.67838848e-19, 9.30704594e-18],
       [1.68627090e-25, 3.85194790e-23],
       [1.41365925e-01, 9.95468211e-05],
       [9.98019457e-12, 3.77562537e-03],
       [7.51175088e-09, 3.90395186e-22]])

#### Calculate the Likelihoods and Point Indicies of the top results

In [4]:
html(calc_top_likelihood_of_points_to_distributions)

In [5]:
points = np.array([[ 2.67536762,  1.61771433],
        [-1.51995242,  8.5575738 ],
        [ 5.9708441 ,  2.88736993],
        [-0.26015255, -1.21744485],
        [-4.24165224,  3.16340135],
        [10.58514974, -1.02516812],
        [11.83292385, -1.69103837],
        [ 2.55579144,  3.80067187],
        [ 1.15372272, -2.601238  ],
        [ 6.47700649,  8.65276974]])

distributions = np.array([
    [[3, 4], [1, 0], [0, 1]],
    [[2, 0], [1, 0], [0, 1]],
    [[3, 4], [1, 0], [0, 1]],
    [[2, 0], [1, 0], [0, 1]]
])

calc_top_likelihood_of_points_to_distributions(distributions, points)

(array([[8.84226101e-03, 3.42373678e-02, 8.84226101e-03, 3.42373678e-02],
        [1.03867726e-03, 9.28095634e-07, 1.03867726e-03, 9.28095634e-07],
        [9.61006664e-10, 5.89820140e-03, 9.61006664e-10, 5.89820140e-03],
        [1.41365925e-01, 9.95468211e-05, 1.41365925e-01, 9.95468211e-05],
        [9.98019457e-12, 3.77562537e-03, 9.98019457e-12, 3.77562537e-03],
        [7.51175088e-09, 3.90395186e-22, 7.51175088e-09, 3.90395186e-22]]),
 array([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]]))